## Creación de una vista estática

Existen dos tipos de views: Basadas en Funciones y Basadas en Clases
- Las funciones se nombran con la primera letra en minúscula y las clases con la primera letra en mayúscula

### Cómo crear una aplicación en Django
```
python manage.py startapp <nombre_de_la_app>

python manage.py startapp ecommerce
```

### ecommerce/views.py

In [ ]:
from django.shortcuts import render
from django.http import HttpResponse

# Create your views here.
def home(request):
    return HttpResponse("<h1>Hola Mundo</h1>")

### ecommerce/urls.py

In [ ]:
from django.urls import path

from ecommerce import views

urlpatterns = [
    path("", views.home, name="home"),
]

### config/urls.py

In [ ]:
from django.conf import settings
from django.contrib import admin
from django.urls import include, path

urlpatterns = [
    path("up/", include("up.urls")),
    path("", include("pages.urls")),
    path("", include("ecommerce.urls")), # <---
    path("admin/", admin.site.urls),
]

## CRUD y Vistas

### C - Create
### R - Read
### U - Update
### D - Delete

Crear Modelos -> Agregar la app a INSTALLED_APPS -> Crear y aplicar migraciones -> Agregar admin

### ecommerce/models.py

In [ ]:
class ProductModel(models.Model):
    title = models.TextField
    price = models.FloatField

### config/settings.py

In [ ]:
# Application definitions
INSTALLED_APPS = [
    "pages.apps.PagesConfig",
    "ecommerce.apps.EcommerceConfig", # <---
    "django.contrib.admin",
    "django.contrib.auth",
    "django.contrib.contenttypes",
    "django.contrib.sessions",
    "django.contrib.messages",
    "django.contrib.staticfiles",
]

### Cómo correr migraciones

In [ ]:
- python manage.py makemigrations
- python manage.py migrate

### Cómo registrar modelos en admin

In [ ]:
from django.contrib import admin
from .models import ProductModel

# Register your models here.
admin.site.register(ProductModel)

### Cómo crear un super user para el admin

In [ ]:
- python manage.py createsuperuser

### Tipos básicos de Vistas

In [ ]:
- List View -  Muestra los usuarios creados
- Create View - Aquí se pueden insertar los datos para crear un usuario
- Retrieve and Update View - Primero trae el registro y después lo expone para ser actualizado 

### ecommerce/views.py

In [ ]:
from django.shortcuts import render
from django.http import HttpResponse, HttpResponseRedirect

from .models import ProductModel

# Create your views here.
def product_model_list_view(request):
    querySet = ProductModel.objects.all()
    print(querySet)
    return HttpResponse("ecommerce personalizado")

## Uso de Templates
### ecommerce/views.py

In [ ]:
from django.shortcuts import render
from django.http import HttpResponse, HttpResponseRedirect

from .models import ProductModel

# Create your views here.
def product_model_list_view(request):
    queryset = ProductModel.objects.all()
    print(queryset)
    template = "ecommerce/list-view.html"
    context = {}
    return redner(request, template, context)

### templates/ecommerce/list-view.html

In [ ]:
<h1>
Template de: Vista de Listado
</h1>

## Usando el contexto
### ecommerce/views.py


In [ ]:
from django.shortcuts import render
from django.http import HttpResponse, HttpResponseRedirect

from .models import ProductModel

# Create your views here.
def product_model_list_view(request):
    queryset = ProductModel.objects.all()
    print(queryset)
    template = "ecommerce/list-view.html"
    context = {
        "products": queryset # <-----
    }
    return render(request, template,context)

### templates/ecommerce/list-view.html

In [ ]:
<h1>
Vista de Listado
</h1>
{% for product in products %}
    <li>
        {{ product.title }} {{ product.price }}
    </li>
{% endfor %}

## Vista de detalle
### ecommerce/views.py

In [ ]:
from django.contrib.auth.decorators import login_required
from django.shortcuts import render, get_object_or_404
from django.http import HttpResponse, HttpResponseRedirect

from.models import ProductModel

# Create your views here.
def product_model_detail_view(request, product_id):
    instance = get_object_or_404(ProductModel, id=product_id)
    context = {
    "product": instance
    }
    template = "ecommerce/detail-view.html"
    return render(request, template, context)

### ecommerce/urls.py

In [ ]:
path("<int:product_id>", views.product_model_details_view, name="details"), # <---

### templates/ecommerce/details-view.html

In [ ]:
<h1>
{{product.title}}
{{product.price}}
</h1>

### templates/ecommerce/list-view.html

In [ ]:
<h1>List View</h1>
{% for product in products %}
    <li>
        <a href="/ecommerce/{{product.id}}">{{product.title}}</a>: {{product.price}}
    </li>
{% endfor %}

## Vista de Creación
### ecommerce/views.py

In [ ]:
from django.contrib.auth.decorators import login_required
from django.contrib import messages
from django.shortcuts import render, get_object_or_404
from django.http import HttpResponse, HttpResponseRedirect

from .forms import ProductModelForm
from .models import ProductModel

# Create your views here
# @login_required
def product_model_create_view(request):
    form = ProductModelForm(request.POST or None)
    if form.is_valid():
        instance = form.save(commit=False)
        instance.save()
        messges.success(request, "Producto creado con éxito")
        return HttpResponseRedirect("/ecommerce/{product_id}".format(product_id=instance.id))
        context = {
            "form": form
        }
        template = "ecommerce/create-view.html"
        return render(request, template, context)
                              

### ecommerce/forms.py
crear archivo si no existe

In [ ]:
from django import forms

from .models import ProductModel

class ProductModelForm(forms.ModelForm):
    class Meta:
        model = ProductModel
        fields = [
            "title",
            "price",
        ]

### ecommerce/messages.html

In [ ]:
{% if messages %}
<ul class="messages">
    {% for message in messages %}
    <li{% if message.tags %} class="{{ message.tags }}"{% endif %}>{{ message }}</li>
    {% endfor %}
</ul>
{% endif %}

### templates/ecommerce/create-view.html

In [ ]:
{% include "ecommerce/messages.html" %} # <--- sustituye esta línea por el contenido del archivo al que hace referencia

<h1>Crear un Producto Nuevo</h1>

<form method="POST" action="."> {% csrf_token %}
{{form.as_p}}

<input type="submit" value="Crear">
</form>

### ecommerce/urls.py

In [ ]:
from django.urls import path

from ecommerce import views

urlpatterns = [
    path("", views.product_model_list_view, name="list"),
    path("<int:product_id>", views.product_model_details_view, name="details"),
    path("create", views.product_model_create_view, name="create"), # <---
]

### ecommerce/views.py

In [ ]:
# @login_required
def product_model_update_view(request, product_id=None):
    instance = get_object_or_404(ProductModel, id=product_id)
    form = ProductModelForm(request.POST or None, instance=instance)
    if form.is_valid():
        instance = form.save(commit=False)
        instance.save()
        messages.success(request, "Producto actualizado con éxito")
        return HttpResponseRedirect("/ecommerce/{product_id}".format(product_id=instance.id))
    context = {
        "form":form
    }
    template = "ecommerce/update-view.html"
    return render(request, template,context)

### templates/ecommerce/update-view.html

In [ ]:
{% include "./messages.html" %}

<h1>Actualizar Producto {{form.instance.title}}</h1>

<form method="POST" action=""> 
    {% csrf_token %}
    {{ form.as_p }}

<input type="submit" value="Actualizar">

</form>

### ecommerce/urls.py

In [ ]:
from django.urls import path

from ecommerce import views

urlpatterns = [
    path("", views.product_model_list_view, name="list"),
    path("<int:product_id>", views.product_model_details_view, name="details"),
    path("create", views.product_model_create_view, name="create"), 
    path("<int:product_id>/edit", views.product_model_update_view, name="update"), # <-----
]

## Vista para Eliminar
### ecommerce/views.py

In [ ]:
from django.contrib.auth.decorators import login_required
from django.contrib import messages
from django.shortcuts import render, get_object_or_404
from django.http import HttpResponse, HttpResponseRedirect

from .forms import ProductModelForm
from .models import ProductModel

# Create your views here
def product_model_delete_view(request, product_id):
    instance = get_object_or_404(ProductModel, id=product_id)
    if request.method == "POST":
        instance.delete()
        HttpResponseRedirect("/ecommerce/")
        messages.success("Producto eliminado con éxito")
    context = {
        "product":instance
    }
    template="/ecommerce/delete-view.html"
    return render(request, template, context)


### template/ecommerce/delete.view.html

In [ ]:
<h1>Eliminar Producto {{product.title}}</h1>

<form method="POST" action="">
    {% csrf_token %}
    ¿Estás seguro que queires este producto?
    <input type="submit" value="Eliminar">
    <a href="/ecommerce/{{product.id}}">Cancelar</a>
</form>

### ecommerce/urls.py

In [ ]:
    path("<int:product_id>/delete", views.product_model_delete_view, name="delete"), # <-----

## Búsqueda dentro de Vista de Listado
### templates/ecommerce/search.html

<form method="GET" action="/ecommerce/">
    <input type="text" name="q" placeholder="Buscar">
</form>

### templates/ecommerce/list-view.html

In [ ]:
{% include "ecommerce/search.html" %}

In [ ]:
def product_model_search_view(request):
    
    context = {
    }
    template = "/search-view.html"
    return render(request, template, context)